In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [8]:
df = pd.read_json("news_link.json")
# df = df.head(10)
df

link   category  \
0       https://www.huffpost.com/entry/covid-boosters-...  U.S. NEWS   
1       https://www.huffpost.com/entry/american-airlin...  U.S. NEWS   
2       https://www.huffpost.com/entry/funniest-tweets...     COMEDY   
3       https://www.huffpost.com/entry/funniest-parent...  PARENTING   
4       https://www.huffpost.com/entry/amy-cooper-lose...  U.S. NEWS   
...                                                   ...        ...   
209522  https://www.huffingtonpost.com/entry/rim-ceo-t...       TECH   
209523  https://www.huffingtonpost.com/entry/maria-sha...     SPORTS   
209524  https://www.huffingtonpost.com/entry/super-bow...     SPORTS   
209525  https://www.huffingtonpost.com/entry/aldon-smi...     SPORTS   
209526  https://www.huffingtonpost.com/entry/dwight-ho...     SPORTS   

             date               authors  
0      2022-09-23  Carla K. Johnson, AP  
1      2022-09-23        Mary Papenfuss  
2      2022-09-23         Elyse Wanshel  
3      2022-09-23      Caroline Bologna  
4      2022-09-22        Nina Golgowski  
...           ...                   ...  
209522 2012-01-28      Reuters, Reuters  
209523 2012-01-28                        
209524 2012-01-28                        
209525 2012-01-28                        
209526 2012-01-28                        

[209527 rows x 4 columns]

In [3]:
df['link'][1]

'https://www.huffpost.com/entry/american-airlines-passenger-banned-flight-attendant-punch-justice-department_n_632e25d3e4b0e247890329fe'

In [6]:
df.isnull().sum()

link        0
category    0
date        0
authors     0
dtype: int64

In [44]:
import aiohttp
import asyncio
from bs4 import BeautifulSoup
import pandas as pd
import nest_asyncio

nest_asyncio.apply()

links_list = df['link'].tolist()
categories_list = df['category'].tolist()

async def fetch_content(session, url):
    timeout = aiohttp.ClientTimeout(total=3000) 
    try:
        async with session.get(url, timeout=timeout) as response:
            return await response.text()
    except asyncio.TimeoutError:
        return None  
    except aiohttp.ClientConnectorError as e:
        return e  

async def scrape_task(link, category, session):
    html = await fetch_content(session, link)
    
    if html is None:
        return {
            'Headline': 'No data (Timeout/Error)',
            'Short Description': 'No data (Timeout/Error)',
            'Author': 'No data (Timeout/Error)',
            'Category': category
        }

    if isinstance(html, aiohttp.ClientConnectorError):
        return {
            'Headline': 'Connection Error',
            'Short Description': 'Connection Error',
            'Author': 'Connection Error',
            'Category': category
        }

    soup = BeautifulSoup(html, "html.parser")

    headline_element = soup.find("h1", class_="headline")
    headline = headline_element.text.strip() if headline_element else 'No headline found'

    short_description_element = soup.find("div", class_="dek")
    short_description = short_description_element.text.strip() if short_description_element else 'No description found'

    author_element = soup.find("div", class_="entry__wirepartner entry-wirepartner")
    author = author_element.text.strip() if author_element else 'No author found'

    return {
        'Headline': headline,
        'Short Description': short_description,
        'Author': author,
        'Category': category
    }

async def main():
    tasks = []
    async with aiohttp.ClientSession() as session:
        for link, category in zip(links_list, categories_list):
            task = asyncio.create_task(scrape_task(link, category, session))
            tasks.append(task)

        results = await asyncio.gather(*tasks, return_exceptions=True)
        return results

loop = asyncio.get_event_loop()
scraped_data = loop.run_until_complete(main())

# Process the scraped data
df_scraped = pd.DataFrame([data if not isinstance(data, Exception) else {} for data in scraped_data])
df_scraped.head()


Headline  \
0  Over 4 Million Americans Roll Up Sleeves For O...   
1  American Airlines Flyer Charged, Banned For Li...   
2  23 Of The Funniest Tweets About Cats And Dogs ...   
3  The Funniest Tweets From Parents This Week (Se...   
4  Woman Who Called Cops On Black Bird-Watcher Lo...   

                                   Short Description            Author  \
0  Health experts said it is too early to predict...  Carla K. Johnson   
1  He was subdued by passengers and crew when he ...   No author found   
2  "Until you have a dog you don't understand wha...   No author found   
3  "Accidentally put grown-up toothpaste on my to...   No author found   
4  Amy Cooper accused investment firm Franklin Te...   No author found   

    Category  
0  U.S. NEWS  
1  U.S. NEWS  
2     COMEDY  
3  PARENTING  
4  U.S. NEWS

In [45]:
df_scraped.to_csv("data.csv", index=False)